In [1]:
from keras.models import load_model, Sequential
from keras.layers import *
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np

In [3]:
pip install tensorflow keras opencv-python numpy

Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/331.7 MB ? eta -:--:--
     --------------------------------------- 0.5/331.7 MB 11.1 MB/s eta 0:00:30
     --------------------------------------- 1.1/331.7 MB 11.2 MB/s eta 0:00:30
     --------------------------------------- 1.6/331.7 MB 11.1 MB/s eta 0:00:30
     --------------------------------------- 2.1/331.7 MB 11.1 MB/s eta 0:00:30
     --------------------------------------- 2.6/331.7 MB 11.0 MB/s eta 0:00:30
     --------------------------------------- 3.1/331.7 MB 11.1 MB/s eta 0:00:30
     --------------------------------------- 3.6/331.7 MB 11.0 MB/s eta 0:00:30
     --------------------------------------- 4.1/331.7 MB 11.4 MB/s eta 0:00:29
      -------------------------------------- 4.7/331.7 MB 11.0 MB/s eta 0:00:30
      -------------------------------------- 5.2/331.7 MB 11.0 


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
face_classifier = cv2.CascadeClassifier("face.xml") #xml file for OpenCV to detect frontal face
model = load_model("checkpoint1.h5") #out pretrained weighted file 

class_labels = ["Angry/Disgiust","Happy","Neutral","Sad","Surprise/Fear"]

cap = cv2.VideoCapture( r"E:\ML\FACE EXPRESSIOON\exp.mp4" ) #your video file here ("0" in case of webcam)

In [ ]:
try:
    while True:
        ret , frame = cap.read() #return valur and the fram
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        #converting to gray as our data was trained using gray images
        
        faces = face_classifier.detectMultiScale( gray , scaleFactor=1.3 , minNeighbors = 5 )

        for x,y,w,h in faces:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,225,225),3) #drawing the rectangle over faces

            roi_gray = gray[y:y+h,x:x+w] #again converting the rectangle portion to gray to make sure
            roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA ) #resizing as per our dataset 48*48

            if np.sum([roi_gray])!=0:
                roi = roi_gray.astype("float")/255.0
                roi = img_to_array(roi)
                roi = np.expand_dims(roi,axis=0)

                pred = model.predict(roi)[0]
                labels = class_labels[pred.argmax()]
                # gets the label from our array
                label_position=(x,y)
                #starting of our rectangle
                cv2.putText(frame,labels,label_position,cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,2,(0,225,0),3)
                #places the text
            else:
                cv2.putText(frame,"No Face Found Sorry!",(20,60),cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,2,"red",3)
                
        cv2.namedWindow('Facial Expression Recognition',cv2.WINDOW_NORMAL)
        cv2.resizeWindow('Facial Expression Recognition', 1000,800)
        cv2.imshow("Facial Expression Recognition",frame)
        #showing the final rendered video
        if cv2.waitKey(1) & 0xFF==ord("q"): #if "Q" is pressed then the video window will be closed
            break
    cap.release()
    cv2.destroyAllWindows()
    #closes the frame window
except:
    print("error occured")
    cap.release()
    cv2.destroyAllWindows()